In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import os
#print(os.listdir("../input"))

In [4]:
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor


In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import gc
import seaborn as sns
import warnings
import scipy.signal as sg
warnings.filterwarnings("ignore")

from scipy import stats

In [27]:
train = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/train.csv', index_col='id')
test = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/test.csv', index_col='id')

In [28]:
display(train.head())

,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
id,,,,,
0,dsgdb9nsd_000001,1,0,1JHC,84.807599999999994
1,dsgdb9nsd_000001,1,2,2JHH,-11.257000000000000
2,dsgdb9nsd_000001,1,3,2JHH,-11.254799999999999
3,dsgdb9nsd_000001,1,4,2JHH,-11.254300000000001
4,dsgdb9nsd_000001,2,0,1JHC,84.807400000000001


In [29]:
test.head()

,molecule_name,atom_index_0,atom_index_1,type
id,,,,
4658147,dsgdb9nsd_000004,2,0,2JHC
4658148,dsgdb9nsd_000004,2,1,1JHC
4658149,dsgdb9nsd_000004,2,3,3JHH
4658150,dsgdb9nsd_000004,3,0,1JHC
4658151,dsgdb9nsd_000004,3,1,2JHC


In [8]:
structures = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/structures.csv')
display(structures.head())

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.0126981359,1.0858041580,0.0080009958
1,dsgdb9nsd_000001,1,H,0.0021504160,-0.0060313176,0.0019761204
2,dsgdb9nsd_000001,2,H,1.0117308430,1.4637511620,0.0002765748
3,dsgdb9nsd_000001,3,H,-0.5408150690,1.4475266140,-0.8766437152
4,dsgdb9nsd_000001,4,H,-0.5238136345,1.4379326440,0.9063972942


In [22]:
structures[structures['atom'] == 'C'][0:10]

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.0126981359,1.0858041580,0.0080009958
12,dsgdb9nsd_000004,0,C,0.5995394918,0.0000000000,1.0000000000
13,dsgdb9nsd_000004,1,C,-0.5995394918,0.0000000000,1.0000000000
16,dsgdb9nsd_000005,0,C,-0.0133239314,1.1324657150,0.0082758861
19,dsgdb9nsd_000007,0,C,-0.0187040036,1.5255820150,0.0104328082
20,dsgdb9nsd_000007,1,C,0.0021037439,-0.0038819102,0.0019988214
27,dsgdb9nsd_000008,0,C,-0.0082881667,1.3904697790,-0.0056006901
33,dsgdb9nsd_000009,0,C,-0.0178210241,1.4643578830,0.0100939689
34,dsgdb9nsd_000009,1,C,0.0020881587,0.0095077743,0.0020119979
35,dsgdb9nsd_000009,2,C,0.0183408510,-1.1918051790,-0.0045050842


In [14]:
dipole_moments = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/dipole_moments.csv')
dipole_moments.head()

,molecule_name,X,Y,Z
0,dsgdb9nsd_000001,0.0000,0.0,0.0000
1,dsgdb9nsd_000002,-0.0002,0.0,1.6256
2,dsgdb9nsd_000003,0.0000,0.0,-1.8511
3,dsgdb9nsd_000005,0.0000,0.0,-2.8937
4,dsgdb9nsd_000007,0.0000,0.0,0.0000


In [15]:
magnetic_shielding = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/magnetic_shielding_tensors.csv')
magnetic_shielding.head()

,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ
0,dsgdb9nsd_000001,0,195.314999999999998,0.0000,-0.0001,0.0000,195.317000000000007,0.0007,-0.0001,0.0007,195.317000000000007
1,dsgdb9nsd_000001,1,31.340999999999998,-1.2317,4.0544,-1.2317,28.954599999999999,-1.7173,4.0546,-1.7173,34.086100000000002
2,dsgdb9nsd_000001,2,31.581399999999999,1.2173,-4.1474,1.2173,28.903600000000001,-1.6036,-4.1476,-1.6036,33.896700000000003
3,dsgdb9nsd_000001,3,31.517199999999999,4.1086,1.2723,4.1088,33.906799999999997,1.6950,1.2724,1.6951,28.957899999999999
4,dsgdb9nsd_000001,4,31.402899999999999,-4.0942,-1.1793,-4.0944,34.077599999999997,1.6259,-1.1795,1.6260,28.901299999999999


In [17]:
mulliken_charges = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/mulliken_charges.csv')
mulliken_charges.head()

,molecule_name,atom_index,mulliken_charge
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [25]:
mulliken_charges[mulliken_charges['molecule_name'] == 'dsgdb9nsd_000007']
#mulliken_charges

,molecule_name,atom_index,mulliken_charge
15,dsgdb9nsd_000007,0,-0.345672
16,dsgdb9nsd_000007,1,-0.345672
17,dsgdb9nsd_000007,2,0.115222
18,dsgdb9nsd_000007,3,0.115225
19,dsgdb9nsd_000007,4,0.115225
20,dsgdb9nsd_000007,5,0.115225
21,dsgdb9nsd_000007,6,0.115222
22,dsgdb9nsd_000007,7,0.115225


In [18]:
potential = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/potential_energy.csv')
potential.head()

,molecule_name,potential_energy
0,dsgdb9nsd_000001,-40.523679999999999
1,dsgdb9nsd_000002,-56.560250000000003
2,dsgdb9nsd_000003,-76.426079999999999
3,dsgdb9nsd_000005,-93.428489999999996
4,dsgdb9nsd_000007,-79.838690000000000


In [26]:
scalar_coupling = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/scalar_coupling_contributions.csv')
scalar_coupling.head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.022400000000005,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.034700000000001,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.032500000000001,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.031900000000000,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.022199999999998,0.254585,1.25861,0.272013


In [9]:
# Map the atom structure data into train and test files

def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

test = map_atom_info(test, 0)
test = map_atom_info(test, 1)

In [9]:
# Engineer a single feature: distance vector between atoms
#  (there's ways to speed this up!)
% time
def dist(row):
    return ( (row['x_1'] - row['x_0'])**2 +
             (row['y_1'] - row['y_0'])**2 +
             (row['z_1'] - row['z_0'])**2 ) ** 0.5

train['dist'] = train.apply(lambda x: dist(x), axis=1)
test['dist'] = test.apply(lambda x: dist(x), axis=1)

In [10]:
molecules = train.pop('molecule_name')
test = test.drop('molecule_name', axis=1)

y = train.pop('scalar_coupling_constant')

# Label Encoding
for f in ['type', 'atom_0', 'atom_1']:
    lbl = LabelEncoder()
    lbl.fit(list(train[f].values) + list(train[f].values))
    train[f] = lbl.transform(list(train[f].values))
    test[f] = lbl.transform(list(test[f].values))

In [16]:
train.head()

,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,1,0,0,0,0.002150416,-0.0060313176,0.0019761204,0,-0.0126981359,1.085804158,0.0080009958
1,1,2,3,0,0.002150416,-0.0060313176,0.0019761204,1,1.0117308430,1.463751162,0.0002765748
2,1,3,3,0,0.002150416,-0.0060313176,0.0019761204,1,-0.5408150690,1.447526614,-0.8766437152
3,1,4,3,0,0.002150416,-0.0060313176,0.0019761204,1,-0.5238136345,1.437932644,0.9063972942
4,2,0,0,0,1.011730843,1.4637511620,0.0002765748,0,-0.0126981359,1.085804158,0.0080009958


In [13]:
yoof = np.zeros(len(train))
yhat = np.zeros(len(test))

n_splits = 3
gkf = GroupKFold(n_splits=n_splits) # we're going to split folds by molecules

fold = 0
for in_index, oof_index in gkf.split(train, y, groups=molecules):
    fold += 1
    print(f'fold {fold} of {n_splits}')
    X_in, X_oof = train.values[in_index], train.values[oof_index]
    y_in, y_oof = y.values[in_index], y.values[oof_index]
    reg = RandomForestRegressor(n_estimators=250,
                                max_depth=9,
                                min_samples_leaf=3,
                                n_jobs=-1)
    reg.fit(X_in, y_in)
    yoof[oof_index] = reg.predict(X_oof)
    yhat += reg.predict(test)

yhat /= n_splits

fold 1 of 3
fold 2 of 3
fold 3 of 3


In [15]:
sample_submission = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/sample_submission.csv', index_col='id')

benchmark = sample_submission.copy()
benchmark['scalar_coupling_constant'] = yhat
benchmark.to_csv('atomic_distance_benchmark.csv')

E:\ANACONDA\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
plot_data = pd.DataFrame(y)
plot_data.index.name = 'id'
plot_data['yhat'] = yoof
plot_data['type'] = pd.read_csv('../input/train.csv', index_col='id', usecols=['id', 'type'])

def plot_oof_preds(ctype, llim, ulim):
        plt.figure(figsize=(6,6))
        sns.scatterplot(x='scalar_coupling_constant',y='yhat',
                        data=plot_data.loc[plot_data['type']==ctype,
                        ['scalar_coupling_constant', 'yhat']]);
        plt.xlim((llim, ulim))
        plt.ylim((llim, ulim))
        plt.plot([llim, ulim], [llim, ulim])
        plt.xlabel('scalar_coupling_constant')
        plt.ylabel('predicted')
        plt.title(f'{ctype}', fontsize=18)
        plt.show()

plot_oof_preds('1JHC', 0, 250)
plot_oof_preds('1JHN', 0, 100)
plot_oof_preds('2JHC', -50, 50)
plot_oof_preds('2JHH', -50, 50)
plot_oof_preds('2JHN', -25, 25)
plot_oof_preds('3JHC', -25, 100)
plot_oof_preds('3JHH', -20, 20)
plot_oof_preds('3JHN', -15, 15)